In [ ]:
%pip install -q geopandas folium

In [ ]:
# Import standard libraries
from io import StringIO

# Third-party libraries
import geopandas as gpd
import requests
import numpy as np
import folium
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib import cm

# Load county borders
county_url = "https://raw.githubusercontent.com/samilaine/hallinnollisetrajat/refs/heads/main/maakuntarajat.json"
county_response = requests.get(county_url)
counties = gpd.read_file(StringIO(county_response.text))

# Initialize folium map
m = folium.Map(location=[64.9667, 25.6667], zoom_start=5, tiles='cartodbpositron')

# Define grid parameters
finland_lat_min, finland_lat_max = 59.5, 70.1
finland_lon_min, finland_lon_max = 19.0, 31.6
grid_res = 0.1

# Create grid
grid_lat = np.arange(finland_lat_min, finland_lat_max, grid_res)
grid_lon = np.arange(finland_lon_min, finland_lon_max, grid_res)

# City data
finland_data = [
    {"city": "Helsinki", "latitude": 60.1695, "longitude": 24.9354, "pop": 684589},
    {"city": "Tampere", "latitude": 61.4978, "longitude": 23.7608, "pop": 260358},
    {"city": "Turku", "latitude": 60.4518, "longitude": 22.2666, "pop": 206035},
    {"city": "Oulu", "latitude": 65.0121, "longitude": 25.4651, "pop": 216194},
    {"city": "Rovaniemi", "latitude": 66.5039, "longitude": 25.7294, "pop": 65738},
    {"city": "Kuopio", "latitude": 62.8910, "longitude": 27.6780, "pop": 125668},
    {"city": "Joensuu", "latitude": 62.6010, "longitude": 29.7639, "pop": 78743},
    {"city": "Jyväskylä", "latitude": 62.2426, "longitude": 25.7473, "pop": 149269},
]

# IDW interpolation function
def idw_interpolate(x, y, values, xi, yi, power=2):
    distances = np.sqrt((xi - x[:, None, None])**2 + (yi - y[:, None, None])**2)
    weights = 1 / (np.where(distances == 0, np.nan, distances) ** power)
    total_weights = np.nansum(weights, axis=0)
    weighted_values = np.nansum(weights * values[:, None, None], axis=0)
    return np.divide(weighted_values, total_weights, where=total_weights != 0)

# Interpolation
x = np.array([city["latitude"] for city in finland_data])
y = np.array([city["longitude"] for city in finland_data])
pop = np.array([city["pop"] for city in finland_data])
zi = idw_interpolate(x, y, pop, grid_lat[:, None], grid_lon[None, :])

# Normalize and color mapping
norm = plt.Normalize(vmin=zi.min(), vmax=zi.max())
colors = cm.viridis(norm(zi))

# Create rectangles
if counties is not None:
    for i, lat in enumerate(grid_lat[:-1]):
        for j, lon in enumerate(grid_lon[:-1]):
            pt = Point(lon, lat)
            if counties.contains(pt).any():
                color = colors[i, j]
                color_hex = '#{:02x}{:02x}{:02x}'.format(int(255 * color[0]), int(255 * color[1]), int(255 * color[2]))
                rectangle = folium.Rectangle(
                    bounds=[[lat, lon], [lat + grid_res, lon + grid_res]],
                    color=color_hex,
                    opacity=0.5,
                    fill=True,
                    fill_color=color_hex,
                    fill_opacity=0.5,
                    weight=0.0
                )
                m.add_child(rectangle)

# Display the map
m